In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tempfile

# Import and prepare data

First we read `data.json`

We then start processing this data so we can prepare it for a DNN Classiffier. We prep 4 columns

- edge_count: Total number of edges
- edge_match: Nodes that we know that share the location of this node
- edge_mutual_match: Nodes that we know share the location of this node AND mutually follows this node
- edge_single_match: Nodes that we know share the location of this node BUT does not know this node

In [3]:
df = pd.read_json('data.json')

locations = list(set(df['location']))
df['edge_count'] =  df['edges'].apply(lambda x: len(x))
df['edge_match'] = df.apply(lambda x: len([1 for y in x['edges'] if df.iloc[y]['location']==x['location']]), axis=1)
df['edge_mutual_match'] = df.apply(lambda x: 
                                    len([1 for y in x['edges'] if 
                                         df.iloc[y]['location']==x['location'] and 
                                         int(x['pk']) in df.iloc[y]['edges']]), 
                                    axis=1
                                   )
df['edge_single_match'] = df.apply(lambda x: 
                                    x['edge_match'] -x['edge_mutual_match'], 
                                    axis=1)
df['location_num'] = df['location'].apply(lambda x: locations.index(x))
df

,edges,location,name,pk,edge_count,edge_match,edge_mutual_match,edge_single_match,location_num
0,"[59, 374, 450, 239]",Eindhoven,Robert Mulloy,0,4,3,1,2,0
1,"[483, 340]",Amsterdam,Frank Salmon,1,2,0,0,0,1
2,"[186, 424]",Amsterdam,Sheila Mandell,2,2,0,0,0,1
3,"[356, 93, 236, 456, 140, 30]",Eindhoven,Barbara Bolf,3,6,4,3,1,0
4,"[123, 38, 439, 7, 17, 109, 476, 394, 196, 307]",Amsterdam,Joseph Patterson,4,10,5,5,0,1
5,"[102, 399, 245, 499, 155, 380, 157, 416]",Eindhoven,Marilyn Spencer,5,8,7,4,3,0
6,"[410, 58, 337, 291, 29, 101]",Eindhoven,Aaron Roberts,6,6,2,2,0,0
7,"[280, 87, 179, 417, 4, 69, 105, 210, 243, 467]",Amsterdam,Marjorie Shipley,7,10,2,2,0,1
8,"[150, 238, 29, 251, 279]",Amsterdam,Kyra Benjamen,8,5,2,1,1,1
9,"[97, 140, 152, 73]",Eindhoven,Jerome Smith,9,4,2,1,1,0


# Start defining our features

In [4]:
feature_edge_count = tf.feature_column.numeric_column("edge_count")
feature_edge_match = tf.feature_column.numeric_column("edge_match")
feature_edge_mutual_match = tf.feature_column.numeric_column("edge_mutual_match")
feature_edge_single_match = tf.feature_column.numeric_column("edge_single_match")
labels = tf.feature_column.categorical_column_with_vocabulary_list("location", set(df['location']))

base_columns = [feature_edge_count, feature_edge_match, feature_edge_mutual_match, feature_edge_single_match]

In [12]:
model_dir = tempfile.mkdtemp()
m = tf.estimator.DNNClassifier(
    model_dir=model_dir,
    feature_columns=base_columns,
    hidden_units=[100, 500, 100],
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpt_zimhdv', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [13]:
def input_fn():
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=df['location_num'],
        num_epochs=None,
        shuffle=True
    )

In [14]:
m.train(input_fn=input_fn(), steps=5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Error reported to Coordinator: <class 'SystemError'>, <built-in function TF_Run> returned a result with an error set
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpt_zimhdv/model.ckpt.


SystemError: <built-in function TF_Run> returned a result with an error set